# Evaluation

In [1]:
import sys
import os

repo_path = os.path.sep.join(os.getcwd().split(os.path.sep)[:-3])
print(repo_path)
sys.path.append(repo_path)
%cd ../../..

/home/loebbert/projects/deepclustering/LMU_Master_Practical_SoSe24
/home/loebbert/projects/deepclustering/LMU_Master_Practical_SoSe24


In [2]:
import pandas as pd
from clustpy.deep.autoencoders import FeedforwardAutoencoder
from practical.DeepClustering.DeepECT.evaluation_pipeline import DatasetType, evaluate

[WARNING] Could not import cv2 in clustpy.data.real_video_data. Please install cv2 by 'pip install opencv-python' if necessary


In [3]:
flat_mnist, hierarchical_mnist = evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.MNIST, seed=42
)

Autoencoder parameters loaded from file.
fitting idec hierarchical...


In [ ]:
flat_usps, hierarchical_usps = evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.USPS, seed=42
)

In [ ]:
flat_fashion, hierarchical_fashion = evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.FASHION_MNIST, seed=42
)

In [ ]:
flat_reuters, hierarchical_reuters = evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.REUTERS, seed=42
)

## Flat Clustering

In [ ]:
flat_combined_df = pd.concat([flat_mnist, flat_usps, flat_fashion, flat_reuters], ignore_index=True)
# Pivot the DataFrame to match the desired format
pivot_df = flat_combined_df.pivot(index='method', columns='dataset', values=['nmi', 'acc', 'ari'])

# Reorder the columns to match the order in the image
pivot_df = pivot_df.reindex(columns=[('nmi', DatasetType.MNIST.value), ('acc', DatasetType.MNIST.value), ('ari', DatasetType.MNIST.value),
                                     ('nmi', DatasetType.USPS.value), ('acc', DatasetType.USPS.value), ('ari', DatasetType.USPS.value),
                                     ('nmi', DatasetType.FASHION_MNIST.value), ('acc', DatasetType.FASHION_MNIST.value), ('ari', DatasetType.FASHION_MNIST.value),
                                     ('nmi', DatasetType.REUTERS.value), ('acc', DatasetType.REUTERS.value), ('ari', DatasetType.REUTERS.value)])


# For Jupyter Notebook display with better formatting
pivot_df.style.set_table_styles([
    {'selector': 'thead th', 'props': [('background-color', '#f7f7f9'), 
                                       ('color', '#333'), 
                                       ('border', '1px solid #ddd')]},
    {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#f9f9f9'), ('color', '#333')]},
    {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#fff'), ('color', '#333') ]}
]).set_caption("Results Table")

## Hierarchical Clustering

In [ ]:
hierarchical_combined = pd.concat([hierarchical_mnist, hierarchical_usps, hierarchical_fashion, hierarchical_reuters], ignore_index=True)
# Pivot the DataFrame to match the desired format
pivot_df = hierarchical_combined.pivot(index='method', columns='dataset', values=['dp', 'lp'])

# Reorder the columns to match the order in the image
pivot_df = pivot_df.reindex(columns=[('dp', DatasetType.MNIST.value), ('lp', DatasetType.MNIST.value), 
                                     ('dp', DatasetType.USPS.value), ('lp', DatasetType.USPS.value),
                                     ('dp', DatasetType.FASHION_MNIST.value), ('lp', DatasetType.FASHION_MNIST.value),
                                     ('dp', DatasetType.REUTERS.value), ('lp', DatasetType.REUTERS.value)])


# For Jupyter Notebook display with better formatting
pivot_df.style.set_table_styles([
    {'selector': 'thead th', 'props': [('background-color', '#f7f7f9'), 
                                       ('color', '#333'), 
                                       ('border', '1px solid #ddd')]},
    {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#f9f9f9'), ('color', '#333')]},
    {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#fff'), ('color', '#333') ]}
]).set_caption("Results Table")